In [30]:
import numpy as np
from my_module import mstools
from my_module import forward_trajectory as fwd
from my_module import mbslib
import csv
import pprint

In [14]:
def run_mbs(nsam, per_site_theta, per_site_rho,
            lsites, selpos,
            n_trajectory, nrep_per_traj,
            path_to_mbs_output, path_to_traj):
    '''
    Args:
        nsam,
        per_site_theta,
        per_site_rho,
        lsites,
        selpos,
        n_trajectory (int): 
        nrep_per_traj (int): 
        path_to_mbs_output (str) : path to mbs output files (w/o extentions)
        path_to_traj (str) : path to trajectory files (w/o extentions)
    '''

    cmd = f'mbs {nsam} -t {per_site_theta} -r {per_site_rho} '
    cmd += f'-s {lsites} {selpos} '
    cmd += f'-f {n_trajectory} {nrep_per_traj} {path_to_traj} '
    cmd += f'> {path_to_mbs_output}'

    mbslib.run_command(cmd)


In [15]:
def make_trajectoryfiles_forward(N0, generation, demography_in_year, t_mutation_in_year, s, h, resolution,
                         n_trajectory, path_to_traj):
    '''
    generate trajectory files

    Args:
        N0 (int):
        generation (int): generation time, years/generation
        demography_in_year (list): demographic history/
        t_mutation_in_year (float): time when mutation arises, in year
        s,
        h,
        resolution,
        n_trajectory (int): number of trajectories
        path_to_traj (str) : path to trajectory files (w/o extentions)

    '''
    for i in range(n_trajectory):

        filename = f'{path_to_traj}_{i}.dat'

        # generate trajectory
        trajectory = fwd.mbs_input(t_mutation_in_year,
                                   demography_in_year,
                                   s, h,
                                   generation, N0, resolution,
                                   'NOTLOST')

        # save
        with open(filename, 'w') as f:
            writer = csv.writer(f, delimiter='\t')
            for freq in trajectory:
                writer.writerow(freq)

In [24]:
def parameter_sets_forward(t_mutation_in_year, sel_advantages, nrep):
    params = dict()

    params['N0'] = 5000
    params['generation'] = 20
    params['demography_in_year'] = [[0, 100 * params['N0'] * params['generation'], params['N0']]]

    # selection coefficients
    params['s'] = 0
    params['h'] = 0.5  # <--- co-dominance
    params['resolution'] = 100

    # number of trajectory
    params['n_trajectory'] = nrep
    # coalescent simulation per trajectory
    params['nrep_per_traj'] = 1

    # number of chromosome
    params['nsam'] = 120
    # length of sequence
    params['lsites'] = 10000
    # position of target site
    params['selpos'] = params['lsites']/2

    # mutation rate per site per generation
    params['per_site_theta'] = 1.0 * 10 ** (-8) * 4 * params['N0']
    # recombination rate per site per generation
    params['per_site_rho'] = 1.0 * 10 ** (-8) * 4 * params['N0']


    params_list = list()
    for s in sel_advantages:
        params['s'] = s
        for t in t_mutation_in_year:
            params['t_mutation_in_year'] = t
            params_list.append(params.copy())

    return params_list

In [17]:
def calc_thetapi_S_thetaw_thetal(ms_seq_data, ms_pos_data):
    """ Calculate theta

     Args:
         ms_seq_data(list(str): ms 0-1 sequence data of one replication
         ms_pos_data(list(float)): SNP position data of one replication

     Return:
         thetapi:
         S:
         thetaw:
         thetal:
         thetah:

     """
    #number of sample
    nsam = len(ms_seq_data)

    #get sequence data in int
    int_site_list = [[int(i) for i in list(j)] for j in ms_seq_data]
    
    #calc theta_l
    #calc number of segregating sites
    l = sum([sum(i) for i in int_site_list])
    #calc average number of segregating sites
    thetal = l/(nsam - 1)

    #calc theta_pi, theta_h
    #calc sum of pairwise differences
    k = 0
    h = 0
    for i in zip(*int_site_list):
        der = sum(i)
        k += der*(nsam-der)
        h += der**2
    #clac_thetapi/h
    thetapi = k*2/(nsam*(nsam-1))
    thetah = h*2/(nsam*(nsam-1))

    #calc theta_w
    #calc number of segregating sites
    S = len(ms_pos_data)
    #calc theta_w
    a = 0
    for j in range(1,nsam):
        a += 1/j
    thetaw = S/a
    
    return thetapi, S, thetaw, thetal, thetah

In [18]:
def calc_H(thetapi, S, thetaw, thetal, thetah, n):
    """ Calculate normalized version of Fay and Wu's H

    Args:
        thetapi:
        S:
        thetaw:
        thetal:
        thetah:
        n: sample size

    Return:
        H: Fay and Wu H
    """
    #calc variation of H
    a = sum([1/i for i in range(1, n)])
    b = sum([1/(i**2) for i in range(1, n)])
    v = (n-2)*thetaw/(6*(n-1)) + (18*n**2*(3*n+2)*(b+1/n**2)-(88*n**3+9*n**2-13*n+6))*(S*(S-1)/(a**2+b))/(9*n*(n-1)**2)
    #return nan if number of segregating sites is too small to calculate H
    if v == 0:
        return np.nan
    else:
        H = (thetapi-thetal)/v**(1/2)
        return H

In [19]:
def calc_D(thetapi, S, thetaw, thetal, thetah, n):
    """ Calculate Tajima's D

    Args:
        thetapi:
        S: number of segregating sites
        thetaw:
        thetal:
        thetah:
        n: sample size

    Return:
         D: Tajima's D
    """
    #calc variation of D
    a1 = 0
    for i in range(1,n):
        a1 += 1/i
    a2 = 0
    for i in range(1, n):
        a2 += 1/i**2
    b1 = (n + 1)/(3*(n-1))
    b2 = 2*(n**2 + n + 3)/(9*n*(n-1))
    c1 = b1 - 1/a1
    c2 = b2 - (n+2)/(a1*n) + a2/a1**2
    e1 = c1/a1
    e2 = c2/(a1**2 + a2)
    C = (e1*S + e2*S*(S-1))**0.5
    if C == 0:
        return np.nan
    else:
        D = (thetapi - thetaw)/C
        return D

In [20]:
def ms(nsam, nreps, theta, r_rate, nsite, filename):
    """ Run ms

    Args:
        nsam: number of chromosomes
        nreps: number of replication
        theta: population mutation parameter per region
        r_rate: population recombination parameter per region
        nsite: length of simulated region
    """
    cmd = "ms {} {} -t {} -r {} {} > {}".format(nsam, nreps, theta, r_rate, nsite, filename)
    mstools.run_command(cmd)  

In [21]:
def base_to_seq(hap, base_pos, base_len):
    """ Produces a string of length 'base_len' with A indicating a fixed base
        C indicating an ancestral base at a polymorphic position and G 
        indicating a derived base. """
    cur_site = 0
    cur_base = 0
    seq_parts = list()
    while cur_site < len(base_pos):
        # fill in the empty space
        seq_parts.append('A' * (base_pos[cur_site] - cur_base))
        if (hap[cur_site] == '0'):
            seq_parts.append('C')
        else:
            seq_parts.append('G')
        cur_base = base_pos[cur_site] + 1
        cur_site += 1
    if cur_base < base_len:
        seq_parts.append('A' * (base_len - cur_base))
    return ''.join(seq_parts)


In [22]:
def ms2fasta(fasta_output, seq):
    # 
    base_pos = [0 + i for i in range(len(seq[0]))]
    base_len = len(seq[0])
    outgroup_seq = 'C'*base_len
    # 
    with open(fasta_output, "w") as f:
        # write outgroup seq
        f.write('>seq{}'.format('0'))
        f.write("\n")
        f.write(outgroup_seq)
        f.write("\n\n")
        # write sample seq
        for m, n in enumerate(seq):
            f.write('>seq{}'.format(m+1))
            f.write("\n")
            line = base_to_seq(n, base_pos, base_len)
            f.write(line)
            f.write("\n\n")

In [54]:
def mbs2ms(mbs_input_file, ms_output_file, nsam, n_traj,
                 per_site_theta, per_site_rho, selpos, lsites):
    # generate file
    with open(ms_output_file, 'w') as f:
        # convert mbs format to ms format
        f.write("ms {} {} -t {} -r {} {}\n\n".format(nsam, n_traj,
                                                     per_site_theta * lsites,
                                                     per_site_rho * lsites, lsites))

        # convert into ms format for each line
        for m in mbslib.parse_mbs_data(mbs_input_file):
            f.write("//\n")
            # write segregating sites
            f.write("segsites: {}\n".format(len(m['pos'])))

            # write position
            f.write("positions: ")
            # convert int to str
            pos_list = [str(i/lsites) for i in m['pos']]
            f.write(" ".join(pos_list))
            f.write("\n")

            # write seq
            f.write("\n".join(m["seq"]))
            f.write("\n\n")

### simulate neutral case 

In [70]:
# run ms to generate sample data
# parameter sets
# initial vales
nsam = 120
nreps = 1
theta = 2
r = 2
lsites = 10000
filename = "results/ms_data.txt"
# run ms 
ms(nsam, nreps, theta, r, lsites, filename)

# convert to fasta format
fasta_output = 'sample_data.fas'
for m in mstools.parse_ms_data(filename):
    seq = m['seq']
    ms2fasta(fasta_output, seq)

In [110]:
### calculate H and D
for m in mstools.parse_ms_data(filename):
    theta = calc_thetapi_S_thetaw_thetal(m['seq'], m['pos'])
    TajiD_inhouse = calc_D(*theta, nsam)
    FWH_inhouse = calc_H(*theta, nsam)
print('TajiD in-house neutral case:', TajiD_inhouse)
print('FWH in-house neutral case:', FWH_inhouse)

TajiD in-house neutral case: -0.6327086308397178
FWH in-house neutral case: 0.18630539898438378


### simulate selective case

In [55]:
# initial values
# in units of year
mutation_ages = [10000]
# selection
sel_advantages = [0.01]
# number of trajectory
number_of_trajectory = 1
# parameters sets
params = parameter_sets_forward(mutation_ages, sel_advantages, number_of_trajectory)[0]

# run mbs to genrate sample sequences
# path to trajectory
#path_to_traj = "results/traj_t{}_s{}".format(params['t_mutation_in_year'], params['s'])

# trajectoryの生成

make_trajectoryfiles_forward(params['N0'], params['generation'],
                              params['demography_in_year'], params['t_mutation_in_year'],
                              params['s'], params['h'], params['resolution'],
                              params['n_trajectory'], path_to_traj)
                              

# mbs output filename
path_to_mbs_output = "sample_data_selective/mbs_nsam{}_t{}_s{}.dat".format(params['nsam'], params['t_mutation_in_year'], params['s'])

# run mbs
run_mbs(params['nsam'], params['per_site_theta'], params['per_site_rho'],
        params['lsites'], params['selpos'],
        params['n_trajectory'], params['nrep_per_traj'],
        path_to_mbs_output, path_to_traj)

# convert mbs output to ms output
ms_output = "ms_nsam{}_t{}_s{}.txt".format(params['nsam'], params['t_mutation_in_year'], params['s'])
mbs2ms(path_to_mbs_output, ms_output, params['nsam'], params['n_trajectory'], 
            params['per_site_theta'], params['per_site_rho'], params['selpos'], params['lsites'])

# convert ms output to fasta format
# convert to fasta format
fasta_output = "fasta_nsam{}_t{}_s{}.fas".format(params['nsam'], params['t_mutation_in_year'], params['s'])
for m in mstools.parse_ms_data(ms_output):
    seq = m['seq']
    ms2fasta(fasta_output, seq)

In [58]:
# mbs output filename
# path_to_mbs_output = "mbs_nsam{}_t{}_s{}.dat".format(params['nsam'], params['t_mutation_in_year'], params['s'])
path_to_mbs_output = 'sample_data_selective/mbs_nsam120_t10000_s0.01.dat'
# in units of year
mutation_ages = [10000]
# selection
sel_advantages = [0.01]
# number of trajectory
number_of_trajectory = 1
# parameters sets
params = parameter_sets_forward(mutation_ages, sel_advantages, number_of_trajectory)[0]
# convert mbs output to ms output
ms_output = "ms_nsam{}_t{}_s{}.txt".format(params['nsam'], params['t_mutation_in_year'], params['s'])
# calculate stats
for m in mbslib.parse_mbs_data(path_to_mbs_output):
    print(m['seq'][0])
    theta = calc_thetapi_S_thetaw_thetal(m['seq'], m['pos'])
    TajiD_inhouse = calc_D(*theta, params['nsam'])
    FWH_inhouse = calc_H(*theta, params['nsam'])

print('TajiD in-house selective case:', TajiD_inhouse)
print('FWH in-house selective case:', FWH_inhouse)

000100000001
TajiD in-house selective case: 1.0312906226156557
FWH in-house selective case: 1.0641892876470052
